# Initialization

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from result_saver import SaverProvider

provider = SaverProvider()

In [3]:
from Scratch import load_calibration_memory

DEVICE = 'ibm_sherbrooke'
OTHER_DATE = '2023-10-30'

all_memories = load_calibration_memory(provider, tobecalib_backend=DEVICE, other_date=OTHER_DATE)

Found jobs for backend ibm_sherbrooke with closest execution date 2023-10-27 08:32:22.841567+00:00.


# Functions:

In [4]:
import numpy as np
from sklearn.mixture import GaussianMixture

def fit_gmm(IQ_data):
    data = IQ_data.flatten().real

    gmm = GaussianMixture(n_components=2, covariance_type='tied', random_state=0)
    gmm.fit(data.reshape(-1, 1))

    return gmm

# Fit for each qubit

In [5]:
gmm_dict = {}
for qubit in all_memories.keys():
    gmm_dict[qubit] = {}
    IQ_data = np.append(all_memories[qubit]['mmr_0'], all_memories[qubit]['mmr_1'], axis=0)

    gmm = fit_gmm(IQ_data)
    mean_0 = gmm.means_[0] if gmm.means_[0] < gmm.means_[1] else gmm.means_[1]
    mean_1 = gmm.means_[0] if gmm.means_[0] > gmm.means_[1] else gmm.means_[1]
    var = gmm.covariances_[0][0]

    gmm_dict[qubit]['mean_0'] = mean_0
    gmm_dict[qubit]['mean_1'] = mean_1
    gmm_dict[qubit]['var'] = var

# takes 3.5s

In [6]:
print(gmm_dict)

{0: {'mean_0': array([-35988268.11234397]), 'mean_1': array([29831861.42491607]), 'var': 80426731404453.55}, 1: {'mean_0': array([-2026262.33895329]), 'mean_1': array([2033755.08677977]), 'var': 1078996869668.0481}, 2: {'mean_0': array([-3137837.5737748]), 'mean_1': array([2491744.95759999]), 'var': 1713647691201.2737}, 3: {'mean_0': array([-2646255.16502781]), 'mean_1': array([2455636.33801872]), 'var': 529993330088.8001}, 4: {'mean_0': array([-3406163.64100285]), 'mean_1': array([2834517.13258404]), 'var': 1836487039186.3093}, 5: {'mean_0': array([-4038120.12497526]), 'mean_1': array([1591644.4144473]), 'var': 1749297620613.5388}, 6: {'mean_0': array([-2191067.16113757]), 'mean_1': array([1150667.56668556]), 'var': 1193333707901.1143}, 7: {'mean_0': array([-4467741.50301164]), 'mean_1': array([2867532.64772645]), 'var': 1460684792960.3074}, 8: {'mean_0': array([-3411489.83793801]), 'mean_1': array([3205845.71898854]), 'var': 1762144543071.507}, 9: {'mean_0': array([-1333702.79381388]

# Decoding

In [7]:
from soft_info import RepCodeIQSimulator

DISTANCE = 7
ROUNDS = 7
_RESETS = False
LOGICAL = 0 # NOT NEEDED FOR EXTREME IQ BCS HARDCODED 0

_is_hex = True
if DEVICE == 'ibmq_mumbai':
    _is_hex = False

# Initialize simulator
simulator = RepCodeIQSimulator(provider, DISTANCE, ROUNDS, DEVICE, _is_hex=_is_hex, _resets = _RESETS, other_date=OTHER_DATE)

Found jobs for backend ibm_sherbrooke with closest execution date 2023-10-27 08:32:22.841567+00:00.
Found jobs for backend ibm_sherbrooke with closest execution date 2023-10-27 08:32:22.841567+00:00.
Searching for ibm_sherbrooke and 23.10.27_07h46_300pts_2std


In [8]:
SHOTS = int(1e4)
NOISE_LIST = [3e-2, 0.8e-2, 1e-2, 3e-2] # [two-qubit-fidelity, reset error, measurement error, idle error]
# NOISE_LIST = None
P_AMBIG = 0.1

IQ_data = simulator.generate_IQ(SHOTS, noise_list=NOISE_LIST)
IQ_data_extreme = simulator.generate_extreme_IQ(SHOTS, P_AMBIG, noise_list=NOISE_LIST)

100%|██████████| 8871/8871 [00:03<00:00, 2869.39it/s]


In [9]:
import pymatching
import stim

model = simulator.stim_circ.detector_error_model(decompose_errors=True)
matching = pymatching.Matching.from_detector_error_model(model)

### SOFT INFO DECODING

In [10]:
from src import cpp_soft_info

_DETAILED = True

matching = pymatching.Matching.from_detector_error_model(model)
result = cpp_soft_info.decode_IQ_shots(matching._matching_graph, IQ_data,
                                           ROUNDS, int(LOGICAL), _RESETS, simulator.qubit_mapping, simulator.grid_dict,
                                           simulator.processed_scaler_dict, p_data=-1, p_mixed=-1,
                                           common_measure=-1, _adv_probs=not _RESETS, _bimodal=_RESETS, merge_strategy = "replace", _detailed=_DETAILED,
                                            p_offset = 1, p_multiplicator = 1, _ntnn_edges = not _RESETS)

matching = pymatching.Matching.from_detector_error_model(model)
result_extreme = cpp_soft_info.decode_IQ_shots(matching._matching_graph, IQ_data_extreme,
                                             ROUNDS, int(LOGICAL), _RESETS, simulator.qubit_mapping, simulator.grid_dict,
                                             simulator.processed_scaler_dict, p_data=-1, p_mixed=-1,
                                             common_measure=-1, _adv_probs=not _RESETS, _bimodal=_RESETS, merge_strategy = "replace", _detailed=_DETAILED,
                                              p_offset = 1, p_multiplicator = 1, _ntnn_edges = not _RESETS)

print("num_errors:", result.num_errors, "out of", len(IQ_data), "shots for _RESETS =", _RESETS)
print("num_errors EXTREME:", result_extreme.num_errors, "out of", len(IQ_data_extreme), "shots for _RESETS =", _RESETS)

num_errors: 132 out of 10000 shots for _RESETS = False
num_errors EXTREME: 127 out of 10000 shots for _RESETS = False


In [11]:
from src import cpp_soft_info

matching = pymatching.Matching.from_detector_error_model(model)
result_gaus = cpp_soft_info.decode_IQ_1Dgauss(matching._matching_graph, IQ_data, ROUNDS, int(LOGICAL), _RESETS,
                                         simulator.qubit_mapping, gmm_dict, _DETAILED)

matching = pymatching.Matching.from_detector_error_model(model)
result_extreme_gaus  = cpp_soft_info.decode_IQ_1Dgauss(matching._matching_graph, IQ_data_extreme, ROUNDS, int(LOGICAL), _RESETS,
                                         simulator.qubit_mapping, gmm_dict, _DETAILED)

print("num_errors:", result_gaus.num_errors, "out of", len(IQ_data), "shots for _RESETS =", _RESETS)
print("num_errors EXTREME:", result_extreme_gaus.num_errors, "out of", len(IQ_data_extreme), "shots for _RESETS =", _RESETS)

p0: 3.74035e-11
p1: 0.271039
p0: 0.173774
p1: 4.08565e-20
p0: 0.320148
p1: 3.69942e-24
p0: 0.970743
p1: 2.09189e-17
p0: 0.133895
p1: 7.10939e-40
p0: 0.440028
p1: 8.69297e-48
p0: 0.99105
p1: 3.59292e-07
p0: 3.74035e-11
p1: 0.271039
p0: 0.994873
p1: 1.55866e-13
p0: 0.173774
p1: 4.08565e-20
p0: 0.860456
p1: 8.53552e-16
p0: 0.320148
p1: 3.69942e-24
p0: 0.990454
p1: 5.54093e-16
p0: 0.970743
p1: 2.09189e-17
p0: 0.687797
p1: 6.32856e-25
p0: 0.133895
p1: 7.10939e-40
p0: 0.519484
p1: 1.48177e-33
p0: 0.440028
p1: 8.69297e-48
p0: 8.93783e-08
p1: 0.929013
p0: 0.99105
p1: 3.59292e-07
p0: 0.297322
p1: 1.34318e-08
p0: 0.994873
p1: 1.55866e-13
p0: 0.838424
p1: 3.28642e-20
p0: 0.860456
p1: 8.53552e-16
p0: 0.969206
p1: 1.97748e-17
p0: 0.990454
p1: 5.54093e-16
p0: 0.769888
p1: 9.36215e-33
p0: 0.687797
p1: 6.32856e-25
p0: 0.126378
p1: 5.60671e-29
p0: 0.519484
p1: 1.48177e-33
p0: 0.697069
p1: 4.70532e-05
p0: 8.93783e-08
p1: 0.929013
p0: 0.9583
p1: 3.54452e-14
p0: 0.297322
p1: 1.34318e-08
p0: 0.982862
p1: 3

KeyboardInterrupt: 

# Plot the results

In [ ]:
print("\nIQ:", result.indices, "\nIQ Gauss:", result_gaus.indices)
print("\nIQ EXTREME:", result_extreme.indices, "\nIQ EXTREME Gauss:", result_extreme_gaus.indices)

In [ ]:
from soft_info import draw_matching_graph


index = 136

fig_size = (4, 13)
fig_size = (8, 8)
scale_factor = 0.5

to_be_result = result
details = to_be_result.error_details[index]
print(f"-----IQ {bool(not (index in to_be_result.indices))} decoding -----")
draw_matching_graph(matching=None, d=DISTANCE, T=ROUNDS, syndromes=details.detection_syndromes,
                        matched_edges=details.matched_edges, figsize=fig_size, scale_factor=scale_factor, edge_list=details.edges)

to_be_result = result_gaus
details = to_be_result.error_details[index]
print(f"-----flat {bool(not (index in to_be_result.indices))} decoding-----")
draw_matching_graph(matching=None, d=DISTANCE, T=ROUNDS, syndromes=details.detection_syndromes,
                        matched_edges=details.matched_edges, figsize=fig_size, scale_factor=scale_factor, edge_list=details.edges)